In [131]:
import numpy as np
import pandas as pd
import os
import cv2
from pose import utils
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [132]:
file_id = '../Videos/friends_hd.npy'
print('Loading images...')
images = np.load(''+file_id)
print('Images loaded success.')

Loading images...
Images loaded success.


# Guardar queries como video

In [133]:
def save_video(indeces, path, df, conocidos=False):
    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    # Define the fps to be equal to 10. Also frame size is passed.

    try:
        out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (images.shape[2],images.shape[1]))
        
        frames_to_write = {}
        
        print("#### PROCESSING ####")
        for frame_id in tqdm(indeces):
            
            try:
                frame = df.loc[frame_id]
            except KeyError:
                continue
            
            try:
                frame_int = int(frame_id)
                to_visit = frame.iterrows()
                
            except TypeError:
                frame_int = int(frame_id[0])
                to_visit = [(frame_id, frame)]

            if frame_int not in frames_to_write:
                frames_to_write[frame_int] = images[frame_int,:,:,:].copy()
                img = frames_to_write[frame_int]
            else:
                img = frames_to_write[frame_int]
                
            for i, row in to_visit:

                try:
                    looking_at_someone = not np.isnan(row['Mira A (ID)'])
                except TypeError:
                    looking_at_someone = True
                
                if looking_at_someone:
                    looking_back = df.loc[(frame_int,  row['Mira A (Index)']), 'Mira A (ID)'] == row.ID
                else:
                    looking_back = False

                if looking_at_someone and looking_back:
                    cv2.rectangle(img, 
                                  (row.bb_x_min, row.bb_y_min), 
                                  (row.bb_x_max, row.bb_y_max),
                                  (0,255,0),thickness=1)

                elif looking_at_someone :
                    cv2.rectangle(img, 
                                  (row.bb_x_min, row.bb_y_min), 
                                  (row.bb_x_max, row.bb_y_max),
                                  (0,255,255),thickness=1)
                else:
                    cv2.rectangle(img, 
                                  (row.bb_x_min, row.bb_y_min), 
                                  (row.bb_x_max, row.bb_y_max),
                                  (0,0,255),
                                  thickness=1)
                if conocidos:
                    nombre = row.ID
                else:
                    nombre = 'P' + row.ID
                cv2.putText(img, 
                            text=nombre, 
                            org=(row.bb_x_min,row.bb_y_min), 
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                            fontScale=0.5, 
                            thickness=1, 
                            lineType=cv2.LINE_AA, 
                            color=(255, 255, 255))

                cv2.putText(img, 
                            text=row.max_emotion, 
                            org=(row.bb_x_min,row.bb_y_max + 10), 
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                            fontScale=0.5, 
                            thickness=1, 
                            lineType=cv2.LINE_AA, 
                            color=(255, 255, 255))

                # Plot headpose
                [yaw, pitch, roll] = (row.yaw, row.pitch, row.roll)
                x_min, y_min, x_max, y_max = [row.bb_x_min, row.bb_y_min, row.bb_x_max, row.bb_y_max]
                box_height = abs(y_max - y_min)
                utils.draw_axis(img, yaw, pitch, roll, tdx = (x_min + x_max) / 2, tdy= (y_min + y_max) / 2, size = box_height/2)
        
        print("#### SAVING ####")
        for f in tqdm(sorted(frames_to_write.keys())):
            out.write(frames_to_write[f])
            
    finally:
        out.release()

# Sin conocer identidad previa

In [178]:
df = pd.read_csv('../Personas/datos_personas.csv', index_col=['Frame', 'Index'])
df.loc[:, 'ID'] = df.ID.astype(str)

## Guardar todo

In [163]:
save_video(list(range(2412)), 'whole_video_sin_conocidos.avi', df)

#### PROCESSING ####



#### SAVING ####


## Personas felices

In [164]:
only_happy = df.loc[df.max_emotion == 'happy'].index.to_flat_index()

In [165]:
save_video(only_happy, 'only_happy.mp4', df)

#### PROCESSING ####



#### SAVING ####


## Persona 1 feliz

In [166]:
only_happy = df.loc[(df.max_emotion == 'happy') & (df.ID == '0')].index.to_flat_index()

In [167]:
save_video(only_happy, 'only_P0_happy.mp4', df)

#### PROCESSING ####



#### SAVING ####


## Persona 0

In [192]:
only_0 = df.loc[df.ID == '0'].index.to_flat_index()

In [193]:
save_video(only_0, 'only_P0.mp4', df)

#### PROCESSING ####



#### SAVING ####


## ¿A quién miran más cuando están alegres?

In [179]:
df.loc[df.max_emotion == 'happy', 'Mira A (ID)'].value_counts()

2.0     22
4.0     14
5.0     14
3.0     13
0.0      8
10.0     5
19.0     2
1.0      2
18.0     1
Name: Mira A (ID), dtype: int64

Como la persona que más miran estando alegres es P2 entonces mostraremos los las escenas cuando alguien mira a P2 para validar

In [188]:
looks_at_2 = (df.loc[:, 'Mira A (ID)'] == 2)
frames = df[looks_at_2].index.get_level_values('Frame')
is_person_2 = df.loc[frames, 'ID'] == '2'

In [189]:
people_look_at_P2 = df.loc[looks_at_2 | is_person_2].index.to_flat_index()
save_video(people_look_at_P2, 'people_look_at_P2.mp4', df)

#### PROCESSING ####



#### SAVING ####


# Conociendo identidad previa

In [151]:
df = pd.read_csv('../Personas/datos_personas_conocidas.csv', index_col=['Frame', 'Index'])
df.loc[:, 'ID'] = df.ID.astype(str)
df.head()

angry   disgust    scared     happy       sad  surprised  \
Frame Index                                                                
0     0      0.060592  0.004396  0.163676  0.105976  0.437814   0.009562   
      1      0.129491  0.001115  0.046344  0.027393  0.084997   0.010530   
1     0      0.035801  0.003362  0.154518  0.099085  0.434491   0.009204   
      1      0.150720  0.001939  0.047514  0.060440  0.087675   0.011068   
2     0      0.029630  0.005719  0.143816  0.153038  0.269821   0.009704   

              neutral max_emotion        yaw      pitch       roll  \
Frame Index                                                          
0     0      0.217985         sad  52.578339 -22.856438  -9.353592   
      1      0.700128     neutral -74.909027  -2.250671  -3.394562   
1     0      0.263538         sad  52.877441 -24.682152 -11.488426   
      1      0.640644     neutral -66.944817  -6.109840  -3.190430   
2     0      0.388271     neutral  56.523651 -31.214188 -16.066132   

             Mira A (Index)  bb_x_min  bb_y_min  bb_x_max  bb_y_max    ID  \
Frame Index                                                                 
0     0                 1.0       500        57       545       127  Ross   
      1                 0.0       349       100       395       168     6   
1     0                 1.0       498        55       543       125  Ross   
      1                 0.0       352       101       395       168     7   
2     0                 1.0       497        54       541       124  Ross   

            Mira A (ID)  
Frame Index              
0     0               6  
      1            Ross  
1     0               7  
      1            Ross  
2     0               8

## Video completo

In [152]:
save_video(list(range(2412)), 'whole_video_con_conocidos.avi', df, conocidos=True)

#### PROCESSING ####



#### SAVING ####


## Ross feliz

In [153]:
only_happy = df.loc[(df.max_emotion == 'happy') & (df.ID == 'Ross')].index.to_flat_index()

In [154]:
save_video(only_happy, 'only_ross_happy.avi', df, conocidos=True)

#### PROCESSING ####



#### SAVING ####


## Ross mira a Chandler

In [157]:
ross_mira_a_chandler = (df.ID == 'Ross') & (df.loc[:, 'Mira A (ID)'] == 'Chandler')
frames = df[ross_mira_a_chandler].index.get_level_values('Frame')
is_chandler = df.loc[frames, 'ID'] == 'Chandler'

In [158]:
people_look_at_P2 = df.loc[ross_mira_a_chandler | is_chandler].index.to_flat_index()
save_video(people_look_at_P2, 'ross_mira_a_chandler.mp4', df, conocidos=True)

#### PROCESSING ####



#### SAVING ####
